In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, load_prompt
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

# prompt = load_prompt("./prompt.yaml")
# prompt.format(country="Germany")

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [("intro", intro), ("example", example), ("start", start)]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrrg! That is a secret!! Arg arg!!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrr!")

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, load_prompt
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache


# set_llm_cache(InMemoryCache())
# set_llm_cache(SQLiteCache("cache.db"))

set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

chat.predict("이탈리안 파스타는 어떻게 만드니?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이탈리안 파스타는 어떻게 만드니?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [5.32s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이탈리안 파스타를 만드는 방법은 다양하지만, 전통적인 방법은 다음과 같습니다:\n\n1. 물에 소금을 넣고 끓인 후 파스타를 넣어서 삶습니다. 삶는 시간은 각 종류마다 다르므로 포장지에 적힌 시간을 참고하세요.\n\n2. 파스타를 삶는 동안, 별도의 팬에 올리브 오일을 데워 양파와 마늘을 볶습니다.\n\n3. 양파와 마늘이 향긋해지면 토마토 소스를 넣고 섞어줍니다. 필요하다면 소금, 후추, 들기름 등으로 맛을 조절하세요.\n\n4. 삶은 파스타를 물기를 제거한 후 소스에 넣고 섞어줍니다.\n\n5. 그릴 된 야채나 고기를 곁들여서 내놓으면 완성입니다.\n\n이 외에도 다양한 파스타 레시피가 있으니 자신이 원하는 스타일에 맞게 변형하여 만들어보세요. 맛있는 이탈리안 파스타 요리를 즐기세요!",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "이탈리안 파스타를 만드는 방법은 다양하지

'이탈리안 파스타를 만드는 방법은 다양하지만, 전통적인 방법은 다음과 같습니다:\n\n1. 물에 소금을 넣고 끓인 후 파스타를 넣어서 삶습니다. 삶는 시간은 각 종류마다 다르므로 포장지에 적힌 시간을 참고하세요.\n\n2. 파스타를 삶는 동안, 별도의 팬에 올리브 오일을 데워 양파와 마늘을 볶습니다.\n\n3. 양파와 마늘이 향긋해지면 토마토 소스를 넣고 섞어줍니다. 필요하다면 소금, 후추, 들기름 등으로 맛을 조절하세요.\n\n4. 삶은 파스타를 물기를 제거한 후 소스에 넣고 섞어줍니다.\n\n5. 그릴 된 야채나 고기를 곁들여서 내놓으면 완성입니다.\n\n이 외에도 다양한 파스타 레시피가 있으니 자신이 원하는 스타일에 맞게 변형하여 만들어보세요. 맛있는 이탈리안 파스타 요리를 즐기세요!'

In [61]:
chat.predict("이탈리안 파스타는 어떻게 만드니?")

'이탈리안 파스타를 만드는 방법은 다양하지만, 전통적인 방법은 다음과 같습니다:\n\n1. 물에 소금을 넣고 끓인 후 파스타를 넣어서 삶습니다. 삶는 시간은 각 종류마다 다르므로 포장지에 적힌 시간을 따르세요.\n\n2. 파스타를 삶는 동안, 별도의 팬에 올리브 오일을 데워 다진 마늘을 볶습니다.\n\n3. 마늘이 익으면 다진 토마토를 넣고 볶습니다. 필요하다면 소금과 후추로 간을 맞춰주세요.\n\n4. 마지막으로 삶은 파스타를 팬에 넣고 골고루 섞어주세요. 파슷타에 소스가 골고루 베이도록 섞어주는 것이 중요합니다.\n\n5. 그릴 또는 오븐에서 구운 새우나 닭고기를 올려서 곁들여 먹으면 더욱 맛있는 이탈리안 파스타 요리가 완성됩니다.\n\n이 외에도 다양한 파스타 요리가 있으니 자신이 좋아하는 재료와 소스를 활용하여 다양한 이탈리안 파스타를 만들어보세요.'